In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode


df = pd.read_csv("prenoms-dpt2022.csv", sep=";")

display(df)
display(df.dtypes)

FileNotFoundError: [Errno 2] No such file or directory: 'prenoms-dpt2022.csv'

In [2]:
df_clean = df.copy()

df_clean = df_clean[(df_clean["preusuel"].str.lower() != "_PRENOMS_RARES".lower()) & (df_clean["annais"] != "XXXX")]
df_clean['annais'] = df_clean['annais'].astype('int')

display(
    len(df_clean[df_clean["preusuel"] == "KEVIN"])
)
df_clean["preusuel"] = df_clean["preusuel"].apply(lambda x: unidecode(x, "utf-8")) 
display(
    len(df_clean[df_clean["preusuel"] == "KEVIN"])
)

3357

4792

In [3]:
# list_dpt_idf = ["75"]
list_dpt_idf = ["95", "93", "91", "92", "75", "60", "77", "78"]
list_dpt_drom = ["971", "972", "973", "974"]
target_year = "2004"
dist_folder = "2023-2024"
top_n_firstnames = 10

In [25]:
# Filtrage
df_filter = df_clean.copy()
list_year = [target_year]
# list_year = ["2002", "2003", "2004"]
df_filter = df_filter[df_filter["annais"].isin(list_year)]
# df_filter = df_filter[df_filter["dpt"] == "95"]

df_filter.reset_index(drop=True, inplace=True)

# df_filter = (
#     df_filter
#         .groupby(['preusuel', 'annais'])
#         .sum()
#         .sort_values(by="nombre", ascending=False)
#         .groupby(['annais'])
#         .head(7)
#         .reset_index()
#         .sort_values(by="annais", ascending=False)
# )
# df_filter = df_filter[df_filter["nombre"] > 15]
df_filter


,sexe,preusuel,annais,dpt,nombre
0,1,AARON,2004,02,4
1,1,AARON,2004,03,3
2,1,AARON,2004,08,4
3,1,AARON,2004,12,3
4,1,AARON,2004,13,11
...,...,...,...,...,...
48797,2,ZOHRA,2004,06,3
48798,2,ZOHRA,2004,13,4
48799,2,ZOHRA,2004,69,4
48800,2,ZOHRA,2004,93,4


In [7]:
df_filter = df_clean.copy()
df_filter = df_filter[df_filter["dpt"].isin(list_dpt_drom)]

df_maryse = df_filter[df_filter["preusuel"] == "MARYSE"]
df_maryse.sort_values(by='nombre', ascending=False)

,sexe,preusuel,annais,dpt,nombre
3263061,2,MARYSE,1963,972,96
3262700,2,MARYSE,1959,972,83
3262519,2,MARYSE,1957,972,80
3262427,2,MARYSE,1956,972,79
3262967,2,MARYSE,1962,972,73
...,...,...,...,...,...
3263956,2,MARYSE,1981,971,3
3261176,2,MARYSE,1942,974,3
3263904,2,MARYSE,1978,973,3
3262968,2,MARYSE,1962,973,3


In [22]:
# Sujet dev

df_filter = df_clean.copy()
min_threshold = 0

df_filter = df_filter[df_filter["annais"] == target_year]
df_final = []

for dpt in list_dpt_idf:
    df_filter_copy = df_filter.copy()
    df_filter_copy = df_filter_copy[df_filter_copy["dpt"] == dpt]
    df_filter_copy = df_filter_copy.groupby(['preusuel', 'sexe', "dpt", "annais"], as_index=False).sum()
    df_filter_copy = df_filter_copy[df_filter_copy["nombre"] > min_threshold]
    
    df_final = df_filter_copy
    df_final.reset_index(inplace=True, drop=True)
    
    df_final.to_json(f'{dist_folder}/dev/naissances-{target_year}-{dpt}.json', orient="records")
    df_final.to_excel(f'{dist_folder}/dev/naissances-{target_year}-{dpt}.xlsx', index=False)


In [34]:
# Sujet crea

years_range = [str(int(target_year) - 10), target_year, str(int(target_year) + 10)]
df_filter_copy = df_clean.copy()
df_filter_copy = df_filter_copy[df_filter_copy['annais'].isin(years_range)]

current_df = df_filter_copy

df_final = (
    current_df
        .groupby(['preusuel', 'sexe', 'annais'], as_index=False)
        .sum()
)

df_final = df_final.groupby(['sexe', 'annais'], as_index=False).apply(lambda grp: grp.nlargest(top_n_firstnames, "nombre"))
df_final.sort_values(by=['sexe', 'annais'], ascending=False, inplace=True)
df_final.reset_index(drop=True, inplace=True)

df_final.to_excel(f'{dist_folder}/crea/top-{top_n_firstnames}-naissances-{"-".join(years_range)}.xlsx', index=False)

In [35]:
# Sujet crea - prénoms épicène
df_filter = df_clean.copy()
min_threshold = 0

df_filter = df_filter[df_filter["annais"] == target_year]

df_filter = (
    df_filter
        .groupby(['preusuel', 'sexe', "annais"], as_index=False)
        .sum()
)

df_epicene = df_filter[df_filter.duplicated('preusuel', keep=False)]

df_top_epicene_all = (
    df_epicene
        .groupby(['preusuel', "annais"], as_index=False)
        .sum()
)

df_top_epicene_all = df_top_epicene_all.nlargest(top_n_firstnames,  "nombre")

df_top_epicene_splited = df_epicene[df_epicene['preusuel'].isin(df_top_epicene_all['preusuel'])]
display(df_top_epicene_splited)

df_top_epicene_splited.to_excel(f'{dist_folder}/crea/top-{top_n_firstnames}-naissances-{list_year[0]}-epicene-france.xlsx', index=False)

,preusuel,sexe,annais,nombre
153,ALIX,1,2004,187
154,ALIX,2,2004,569
241,ANDREA,1,2004,181
242,ANDREA,2,2004,821
542,CAMILLE,1,2004,369
543,CAMILLE,2,2004,5286
900,ELISE,1,2004,5
901,ELISE,2,2004,1672
1934,LOAN,1,2004,338
1935,LOAN,2,2004,239


In [34]:
# Test

df_filter = df_clean.copy()
df_filter = df_filter[df_filter["sexe"] == 1]
df_filter = df_filter[df_filter["annais"] == target_year]

df_filter[df_filter["preusuel"] == "NOA"]["nombre"].sum()

1317

In [ ]:
# Sujet crea -wip

years_range = ['1994', "2004", "2014"]
df_filter_copy = df_clean.copy()
df_filter_copy = df_filter_copy[df_filter_copy['annais'].isin(
    years_range)]

for dpt in list_dpt_idf:
    current_df = df_filter_copy[df_filter_copy["dpt"] == dpt]

    df_final = (
        current_df
            .sort_values(by="annais", ascending=True)
            .groupby(['sexe', 'annais'], as_index=False).apply(lambda x: x.nlargest(top_n_firstnames, 'nombre'))
    )
    df_final.drop("dpt", inplace=True, axis=1)
    df_final.reset_index(drop=True, inplace=True)
    df_final.sort_values(by=['sexe', 'annais'], ascending=False, inplace=True)
  
    df_final.to_json(f'{dist_folder}/crea/top-{top_n_firstnames}-naissances-{list_year[0]}-{dpt}.json', orient="records")
    df_final.to_excel(f'{dist_folder}/crea/top-{top_n_firstnames}-naissances-{list_year[0]}-{dpt}.xlsx', index=False)